# Demo de Simulación de Balance - Cantidad de Inversiones

## Configuración inicial de librerías

In [1]:
# Contiene las clases Operacion, Caja y Banco.
import simula_balance as sb

# Contiene las reglas de renovación, por ejemplo,
# toda la caja a overnight.
import reglas_renovacion as rn

# Contiene las métricas, por ejemplo, el C46.
import metricas as met

# Se usa pandas sólo para desplegar claramente el gap de liquidez.
import pandas as pd

## t = 0: Configuración inicial del Banco

In [2]:
# Monto de capital
capital = 10.0

# Sobre este objeto caen los vencimientos de activos y pasivos.
# De aquí se obtiene el dinero para captaciones y colocaciones,
# toda operación, al iniciarse, tiene su correspondiente movimiento
# en la caja.
caja = sb.Caja()

# Construimos un Banco muy sencillo con una colocación bullet
# y una captación (también bullet).
# Lo primero son las tasas de mercado
mercado = {'on_col': 0.0, 'on_cap':0.0,
           'depo2':0.0312, 'depo3':0.033, 'depo4':0.0348,
           'inversion':0.027, 'colocacion':0.045}

# Variable que establece los plazos y porcentajes de renovación a cada plazo.
# Si tenemos x:y significa que al plazo x (medido en número de períodos)
# se hace un y% del monto necesario.
plazos = {2:50, 4:50}


# Se construye el crédito
credito = sb.Operacion(mercado['colocacion'], 0, 12, 100, {'id':'1', 'ap':'activo', 'producto':'credito'})

# En esta lista se almacenarán las operaciones que insertaremos inicialmente en el banco.
operaciones = [credito]

# Se construye la mezcla de depósitos
i = 0
for plazo in plazos:
    i += 1
    codigo_tasa = 'depo' + str(plazo)
    ponderador = plazos[plazo] / 100.0
    operaciones.append(sb.Operacion(mercado[codigo_tasa], 0, plazo, ponderador * 90,
                                    {'id':str(i), 'ap':'pasivo', 'producto':'depo'}))
    
# Se da de alta el banco inicial
banco = sb.Banco(caja, operaciones, capital)

### Se construye y muestra el gap de liquidez inicial del banco.

In [3]:
# Se determina el gap de liquidez del Banco.
gap = met.construye_gap_liquidez(banco)

# El gap de liquidez se guarda en un dataframe para imprimirlo bien.
pd_gap = pd.DataFrame([gap()])

# Se muestra el resultado
print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
print pd_gap
print 'c46: ', met.c46(banco, True)

Gap de Liquidez a tiempo t =  0
    0   1       2   3       4   5   6   7   8   9   10  11     12  13  14  15  \
0  0.0   0 -45.702   0 -46.566   0   0   0   0   0   0   0  113.5   0   0   0   

   16  
0   0  
c46:  -0.0


### Se muestra el estado de resultados inicial.

In [4]:
# Se calcula y muestra, no se almacena.
banco.estado_resultados()

{'activo': {'intereses_devengados': 0.0, 'intereses_recibidos': 0.0},
 'pasivo': {'intereses_devengados': 0.0, 'intereses_pagados': 0.0}}

### Se muestra el balance inicial.

In [5]:
# Se calcula y muestra, no se almacena.
banco.balance()

{'activo': {'caja': 0.0, 'credito': 100.0},
 'pasivo': {'depo': -90.0, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

Vamos a definir algunas funciones auxiliares que harán menos verboso el ejercicio:

In [6]:
def muestra_gap():
    # se determina la situación de liquidez y
    gap = met.construye_gap_liquidez(banco)
    pd_gap = pd.DataFrame([gap()])

    # se muestra el resultado.
    print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
    print pd_gap
    print 'c46: ', met.c46(banco, True)
    return

def muestra_resultados():
    # Calcula y muestra el estado de resultados
    er = banco.estado_resultados()
    resultado = 0.0
    for key in er:
        print key, er[key]
        if key == 'activo':
            for interes in er[key]:
                resultado += er[key][interes]
        else:
            for interes in er[key]:
                resultado -= er[key][interes]
    print "resultado: ", resultado
    return

def muestra_balance():
    # Calcula y muestra el balance
    bal = banco.balance()
    for key in bal:
        print key, bal[key]
    return

def muestra_operaciones(t):
    # Muestra los flujos de las operaciones del banco
    activos = banco.activos()
    print 'Activos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for act in activos:
        print act.atributos()['producto'], act.vencimiento(), act.vencimiento() - t, act.monto(), act.interes_devengado(t), act.flujo()
        
    pasivos = banco.pasivos()
    print 'Pasivos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for pas in pasivos:
        print pas.atributos()['producto'], pas.vencimiento(), pas.vencimiento() - t, pas.monto(), pas.interes_devengado(t), pas.flujo()

def avanza():
    # Se avanza un período en el tiempo,
    banco.avanza()

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    
    print
    
    # Se calcula y muestra el EERR.
    muestra_resultados()
    
    print
    
    # Se calcula y muestra el balance
    muestra_balance()
        
    print
    
    # Se muestran las operaciones
    muestra_operaciones(banco.tiempo_actual())
    return
    
def renueva_depo():
    # Se renueva el depósito
    rn.renueva_depos(banco, plazos)

    # se determina la nueva situación de liquidez y
    # se muestra
    muestra_gap()
    return

def toma_inversiones():
    # Se renueva la inversión
    rn.inversion(banco)

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    return

def arregla_c46():
    # Se arregla el C46,
    rn.cumple_c46(banco, plazos, True)

    # Se muestra gap
    muestra_gap()

In [7]:
muestra_operaciones(0)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 12.0 100.0 0.0 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 2.0 45.0 0.0 45.702
depo 4.0 4.0 45.0 0.0 46.566


## t = 1: Se avanza un período en el tiempo y se muestra la nueva situación de liquidez, el EERR, el balance y las operaciones (vigentes y vencidas).

In [8]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  1
    0       1   2       3   4   5   6   7   8   9   10     11  12  13  14  15  \
0  0.0 -45.702   0 -46.566   0   0   0   0   0   0   0  113.5   0   0   0   0   

   16  
0   0  
c46:  45.702

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.7424999999999999}
resultado:  0.3825

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 101.125}
pasivo {'depo': -90.7425, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 11.0 100.0 1.125 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 1.0 45.0 0.351 45.702
depo 4.0 3.0 45.0 0.3915 46.566


### Se "arregla" el C46. Se compran inversiones financiándolas con depósitos a los plazos indicados en la variable **plazos**.

In [9]:
# Se arregla el C46.
arregla_c46()

Gap de Liquidez a tiempo t =  1
    0       1        2       3        4   5   6   7   8   9   10     11  12  \
0  0.0  0.3085 -23.2075 -46.566 -23.6462   0   0   0   0   0   0  113.5   0   

   13  14  15  16  
0   0   0   0   0  
c46:  -0.3085


In [10]:
muestra_operaciones(1)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 11.0 100.0 1.125 113.5
inversion 2.0 1.0 45.702 0.0 46.0105
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 1.0 45.0 0.351 45.702
depo 4.0 3.0 45.0 0.3915 46.566
depo 3.0 2.0 22.851 0.0 23.2075
depo 5.0 4.0 22.851 0.0 23.6462


In [11]:
muestra_resultados()

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.7424999999999999}
resultado:  0.3825


## t = 2: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [12]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  2
       0        1       2        3   4   5   6   7   8   9      10  11  12  \
0  0.3085 -23.2075 -46.566 -23.6462   0   0   0   0   0   0  113.5   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  22.899

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.3085}
pasivo {'intereses_pagados': 0.702, 'intereses_devengados': 1.1600000000000001}
resultado:  0.6965

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.3085, 'credito': 102.25}
pasivo {'depo': -91.86200000000001, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 10.0 100.0 2.25 113.5
inversion 2.0 0.0 45.702 0.0 46.0105
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 45.0 0.0 45.702
depo 4.0 2.0 45.0 0.783 46.566
depo 3.0 1.0 22.851 0.1782 23.2075
depo 5.0 3.0 22.851 0.1988 23.6462


In [14]:
# Se arregla el C46
arregla_c46()

Gap de Liquidez a tiempo t =  2
       0       1        2        3        4   5   6   7   8   9      10  11  \
0  0.0001 -0.1539 -58.0375 -23.6462 -11.6884   0   0   0   0   0  113.5   0   

   12  13  14  15  16  
0   0   0   0   0   0  
c46:  0.1538


In [15]:
# Notar que el nuevo depo es por un monto menor a la inversión porque partimos con caja > 0.
muestra_operaciones(2)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 10.0 100.0 2.25 113.5
inversion 2.0 0.0 45.702 0.0 46.0105
inversion 3.0 1.0 22.899 0.0 23.0536
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 45.0 0.0 45.702
depo 4.0 2.0 45.0 0.783 46.566
depo 3.0 1.0 22.851 0.1782 23.2075
depo 5.0 3.0 22.851 0.1988 23.6462
depo 4.0 2.0 11.2953 0.0 11.4715
depo 6.0 4.0 11.2953 0.0 11.6884


### Veamos como queda el balance.

In [16]:
# Se muestra el balance
banco.balance()

{'activo': {'caja': 0.0001, 'credito': 102.25, 'inversion': 22.899},
 'pasivo': {'depo': -114.4526, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

In [17]:
muestra_resultados()

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.3085}
pasivo {'intereses_pagados': 0.702, 'intereses_devengados': 1.1600000000000001}
resultado:  0.6965


In [18]:
# Se muestra el ratio de inv / col.
banco.inv_sobre_col()

0.2239511002444988

## t = 3: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [19]:
# Se avanza un período en el tiempo
avanza()

Gap de Liquidez a tiempo t =  3
       0        1        2        3   4   5   6   7   8      9   10  11  12  \
0 -0.1538 -58.0375 -23.6462 -11.6884   0   0   0   0   0  113.5   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  58.1913

activo {'intereses_devengados': 3.375, 'intereses_recibidos': 0.46309999999999996}
pasivo {'intereses_pagados': 1.0585, 'intereses_devengados': 1.7585000000000002}
resultado:  1.0211

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 103.375}
pasivo {'depo': -92.20009999999999, 'sobregiro': -0.1538}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 9.0 100.0 3.375 113.5
inversion 2.0 -1.0 45.702 0.0 46.0105
inversion 3.0 0.0 22.899 0.0 23.0536
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 45.0 0.0 45.702
depo 4.0 1.0 45.0 1.1745 46.566
depo 3.0 0.0 22.851 0.0 23.2075
depo 5.0 2.0 22.851 0.3976 23.6462
depo 4.0 1.0 11.2953 0.0881 11.4715
depo 6.0 3.0 11.2953 0.0983 11.6884


### Se arregla el C46.

In [20]:
# Se arregla el C46,
arregla_c46()

Gap de Liquidez a tiempo t =  3
       0       1        2        3        4   5   6   7   8      9   10  11  \
0 -0.0001  0.5466 -53.2738 -11.6884 -30.1877   0   0   0   0  113.5   0   0   

   12  13  14  15  16  
0   0   0   0   0   0  
c46:  -0.5465


In [21]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 9.0 100.0 3.375 113.5
inversion 2.0 -1.0 45.702 0.0 46.0105
inversion 3.0 0.0 22.899 0.0 23.0536
inversion 4.0 1.0 58.1913 0.0 58.5841
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 45.0 0.0 45.702
depo 4.0 1.0 45.0 1.1745 46.566
depo 3.0 0.0 22.851 0.0 23.2075
depo 5.0 2.0 22.851 0.3976 23.6462
depo 4.0 1.0 11.2953 0.0881 11.4715
depo 6.0 3.0 11.2953 0.0983 11.6884
depo 5.0 2.0 29.1725 0.0 29.6276
depo 7.0 4.0 29.1725 0.0 30.1877


In [22]:
muestra_resultados()

activo {'intereses_devengados': 3.375, 'intereses_recibidos': 0.46309999999999996}
pasivo {'intereses_pagados': 1.0585, 'intereses_devengados': 1.7585000000000002}
resultado:  1.0211


## t = 4. Volvemos a avanzar.

In [23]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  4
       0        1        2        3   4   5   6   7      8   9   10  11  12  \
0  0.5465 -53.2738 -11.6884 -30.1877   0   0   0   0  113.5   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  52.7273

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 0.8558999999999999}
pasivo {'intereses_pagados': 2.8007, 'intereses_devengados': 1.2742}
resultado:  1.281

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.5465, 'credito': 104.5}
pasivo {'depo': -93.7655, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 8.0 100.0 4.5 113.5
inversion 2.0 -2.0 45.702 0.0 46.0105
inversion 3.0 -1.0 22.899 0.0 23.0536
inversion 4.0 0.0 58.1913 0.0 58.5841
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -2.0 45.0 0.0 45.702
depo 4.0 0.0 45.0 0.0 46.566
depo 3.0 -1.0 22.851 0.0 23.2075
depo 5.0 1.0 22.851 0.5964 23.6462
depo 4.0 0.0 11.2953 0.0 11.4715
depo 6.0 2.0 11.2953 0.1965 11.6884
depo 5.0 1.0 29.172

### Se arregla el C46.

In [24]:
arregla_c46()

Gap de Liquidez a tiempo t =  4
    0       1        2        3        4   5   6   7      8   9   10  11  12  \
0 -0.0 -0.1906 -38.1858 -30.1877 -26.9983   0   0   0  113.5   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  0.1906


In [25]:
muestra_resultados()

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 0.8558999999999999}
pasivo {'intereses_pagados': 2.8007, 'intereses_devengados': 1.2742}
resultado:  1.281


In [26]:
muestra_balance()

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 52.7273, 'caja': -0.0, 'credito': 104.5}
pasivo {'depo': -145.9463, 'sobregiro': 0.0}


## t = 5: Volvemos a avanzar.

In [27]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  5
       0        1        2        3   4   5   6      7   8   9   10  11  12  \
0 -0.1906 -38.1858 -30.1877 -26.9983   0   0   0  113.5   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  38.3764

activo {'intereses_devengados': 5.625, 'intereses_recibidos': 1.2117999999999998}
pasivo {'intereses_pagados': 4.051, 'intereses_devengados': 1.2329}
resultado:  1.5529

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 105.625}
pasivo {'depo': -93.88149999999999, 'sobregiro': -0.1906}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 7.0 100.0 5.625 113.5
inversion 2.0 -3.0 45.702 0.0 46.0105
inversion 3.0 -2.0 22.899 0.0 23.0536
inversion 4.0 -1.0 58.1913 0.0 58.5841
inversion 5.0 0.0 52.7273 0.0 53.0832
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -3.0 45.0 0.0 45.702
depo 4.0 -1.0 45.0 0.0 46.566
depo 3.0 -2.0 22.851 0.0 23.2075
depo 5.0 0.0 22.851 0.0 23.6462
depo 4.0 -1.0 11.2953 0.0 11.4715


In [28]:
# Arregla el C46
arregla_c46()

Gap de Liquidez a tiempo t =  5
    0       1       2        3        4   5   6      7   8   9   10  11  12  \
0 -0.0  0.4496 -49.772 -26.9983 -19.9546   0   0  113.5   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  -0.4496


## t = 6: Volvemos a avanzar.

In [29]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  6
       0       1        2        3   4   5      6   7   8   9   10  11  12  \
0  0.4496 -49.772 -26.9983 -19.9546   0   0  113.5   0   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  49.3224

activo {'intereses_devengados': 6.75, 'intereses_recibidos': 1.4707999999999997}
pasivo {'intereses_pagados': 4.8511, 'intereses_devengados': 1.5336}
resultado:  1.8361

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.4496, 'credito': 106.75}
pasivo {'depo': -95.36350000000002, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 6.0 100.0 6.75 113.5
inversion 2.0 -4.0 45.702 0.0 46.0105
inversion 3.0 -3.0 22.899 0.0 23.0536
inversion 4.0 -2.0 58.1913 0.0 58.5841
inversion 5.0 -1.0 52.7273 0.0 53.0832
inversion 6.0 0.0 38.3764 0.0 38.6354
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -4.0 45.0 0.0 45.702
depo 4.0 -2.0 45.0 0.0 46.566
depo 3.0 -3.0 22.851 0.0 23.2075
depo 5.0 -1.0 22.851 0.0 23.6462

In [30]:
arregla_c46()

Gap de Liquidez a tiempo t =  6
    0       1        2        3        4   5      6   7   8   9   10  11  12  \
0 -0.0 -0.1167 -51.8159 -19.9546 -25.2868   0  113.5   0   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  0.1167


## t = 7: Volvemos a avanzar

In [31]:
avanza()

Gap de Liquidez a tiempo t =  7
       0        1        2        3   4      5   6   7   8   9   10  11  12  \
0 -0.1167 -51.8159 -19.9546 -25.2868   0  113.5   0   0   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  51.9326

activo {'intereses_devengados': 7.875, 'intereses_recibidos': 1.8036999999999996}
pasivo {'intereses_pagados': 6.1671, 'intereses_devengados': 1.4197000000000002}
resultado:  2.0919

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 107.875}
pasivo {'depo': -95.6664, 'sobregiro': -0.1167}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 5.0 100.0 7.875 113.5
inversion 2.0 -5.0 45.702 0.0 46.0105
inversion 3.0 -4.0 22.899 0.0 23.0536
inversion 4.0 -3.0 58.1913 0.0 58.5841
inversion 5.0 -2.0 52.7273 0.0 53.0832
inversion 6.0 -1.0 38.3764 0.0 38.6354
inversion 7.0 0.0 49.3224 0.0 49.6553
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -5.0 45.0 0.0 45.702
depo 4.0 -3.0 45.0 0.0 46.566
depo 3.0 -4.0 22.8

In [32]:
arregla_c46()

Gap de Liquidez a tiempo t =  7
       0       1        2        3        4      5   6   7   8   9   10  11  \
0  0.0001  0.4672 -46.3853 -25.2868 -26.9304  113.5   0   0   0   0   0   0   

   12  13  14  15  16  
0   0   0   0   0   0  
c46:  -0.4673


## t = 8. Última iteración.

In [33]:
avanza()

Gap de Liquidez a tiempo t =  8
       0        1        2        3      4   5   6   7   8   9   10  11  12  \
0  0.4673 -46.3853 -25.2868 -26.9304  113.5   0   0   0   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  45.918

activo {'intereses_devengados': 9.0, 'intereses_recibidos': 2.1541999999999994}
pasivo {'intereses_pagados': 7.456199999999999, 'intereses_devengados': 1.3578999999999999}
resultado:  2.3401

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.4673, 'credito': 109.0}
pasivo {'depo': -97.12719999999999, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 4.0 100.0 9.0 113.5
inversion 2.0 -6.0 45.702 0.0 46.0105
inversion 3.0 -5.0 22.899 0.0 23.0536
inversion 4.0 -4.0 58.1913 0.0 58.5841
inversion 5.0 -3.0 52.7273 0.0 53.0832
inversion 6.0 -2.0 38.3764 0.0 38.6354
inversion 7.0 -1.0 49.3224 0.0 49.6553
inversion 8.0 0.0 51.9326 0.0 52.2831
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -6.0 45.0 0.0 45.

In [34]:
arregla_c46()

Gap de Liquidez a tiempo t =  8
       0       1        2        3        4   5   6   7   8   9   10  11  12  \
0 -0.0001 -0.1574 -48.3666 -26.9304  89.9839   0   0   0   0   0   0   0   0   

   13  14  15  16  
0   0   0   0   0  
c46:  0.1575


In [35]:
muestra_resultados()

activo {'intereses_devengados': 9.0, 'intereses_recibidos': 2.1541999999999994}
pasivo {'intereses_pagados': 7.456199999999999, 'intereses_devengados': 1.3578999999999999}
resultado:  2.3401
